In [ ]:
项目描述：随机给两个电影，判断用户对电影的喜好
贝叶斯个性化排序是将排序问题转化为序列对：(i>j>z) ==> (i,j),(j,z),
训练这两个数据集就可以得到正确的排序
原理的参考这篇博文：https://www.cnblogs.com/pinard/p/9163481.html

1.下载movielens 1M数据，根据打分构造样本

In [1]:
import numpy
import tensorflow as tf
import os
import random
from collections import defaultdict

def load_data(data_path):
    #
    train_data = defaultdict(list)
    test_data = defaultdict(list)
    watch_list = defaultdict(set)
    max_uid = -1
    max_item = -1
    
    with open(data_path, 'r') as f:
        for line in f.readlines():
            u, i, r, t = map(int,line.strip().split("::"))
            watch_list[u].add(i)
            # 每个用户的评分都存在dict
            # 划分训练集和测试集，测试集为用户最早的两条打分记录
            if len(test_data[u]) == 0:
                test_data[u].append((i,r))
            elif len(test_data[u]) == 1 and test_data[u][0][1] != r:
                test_data[u].append((i,r))
            else:
                train_data[u].append((i,r))
            if u > max_uid:
                max_uid = u
            if i > max_item:
                max_item = i
    return train_data,test_data,watch_list,max_uid,max_item

train_data, test_data, watch_list, user_count, item_count = load_data('ml-1m/ratings.dat')
print(len(train_data), len(test_data), user_count, item_count)
print(train_data[1])
print(test_data[1])

6040 6040 6040 3952
[(914, 3), (3408, 4), (2355, 5), (1197, 3), (1287, 5), (2804, 5), (594, 4), (919, 4), (595, 5), (938, 4), (2398, 4), (2918, 4), (1035, 5), (2791, 4), (2687, 3), (2018, 4), (3105, 5), (2797, 4), (2321, 3), (720, 3), (1270, 5), (527, 5), (2340, 3), (48, 5), (1097, 4), (1721, 4), (1545, 4), (745, 3), (2294, 4), (3186, 4), (1566, 4), (588, 4), (1907, 4), (783, 4), (1836, 5), (1022, 5), (2762, 4), (150, 5), (1, 5), (1961, 5), (1962, 4), (2692, 4), (260, 4), (1028, 5), (1029, 5), (1207, 4), (2028, 5), (531, 4), (3114, 4), (608, 4), (1246, 4)]
[(1193, 5), (661, 3)]


2.随机采样，生成batch训练数据

In [2]:
def generate_train_batch(rating_data, batch_size=256):
    t = []
    for b in range(batch_size):
        u = random.sample(rating_data.keys(), 1)[0]
        i,r1 = random.sample(rating_data[u], 1)[0]
        j,r2 = random.sample(rating_data[u], 1)[0] 
        # 直到选到一个打分不同的pair
        while r1 == r2:
            u = random.sample(rating_data.keys(), 1)[0]
            i,r1 = random.sample(rating_data[u], 1)[0]
            j,r2 = random.sample(rating_data[u], 1)[0]
        #i>j
        if r1 > r2:
            t.append([u, i, j])
        else:
            t.append([u, j, i])
    return numpy.asarray(t)
batch_data = generate_train_batch(train_data)
# [user,moive1,moive2] 但是rat(moive1) > rat(moive2)
print(batch_data[:5])

[[5400 3623 2890]
 [  74  858   34]
 [2925  886 3895]
 [5577 3685 3668]
 [2920  902 1544]]


In [3]:
def generate_test_batch(rating_data):
    t = []
    for u in rating_data:
        i,r1 = rating_data[u][0]
        j,r2 = rating_data[u][1]
        if r1 > r2:
            t.append([u, i, j])
        else:
            t.append([u, j, i])
    return numpy.asarray(t)
batch_data = generate_test_batch(test_data)
# [user,moive1,moive2] 但是rat(moive1) > rat(moive2)
print(batch_data[:5])

[[   1 1193  661]
 [   2 1357 3068]
 [   3 3421 1641]
 [   4 3468 1210]
 [   5 1175 2987]]


3.定义网络结构

In [4]:
regulation_rate = 0.01
bias_reg = 0.01

def bpr_mf(user_count, item_count, hidden_dim):
    u = tf.placeholder(tf.int32, [None])
    i = tf.placeholder(tf.int32, [None])
    j = tf.placeholder(tf.int32, [None])

    with tf.device("/cpu:0"):
        # Uv*Iv + Ub +Ib
        # 定义 用户、物品、偏置的 Embedding 矩阵 既 bpr 中的 W 和H
        user_vec = tf.get_variable("user_vec", [user_count+1, hidden_dim], 
                            initializer=tf.random_normal_initializer(0, 0.1))
        item_vec = tf.get_variable("item_vec", [item_count+1, hidden_dim], 
                                initializer=tf.random_normal_initializer(0, 0.1))
        item_bias = tf.get_variable("item_bias", [item_count+1, 1], initializer=tf.random_normal_initializer(0, 0.1))    #item bias
        
        # 取出 Embeding 后的用户向量
        u_vec = tf.nn.embedding_lookup(user_vec, u)
        i_vec = tf.nn.embedding_lookup(item_vec, i)
        j_vec = tf.nn.embedding_lookup(item_vec, j)  
        #
        i_bias = tf.nn.embedding_lookup(item_bias, i)       
        j_bias = tf.nn.embedding_lookup(item_bias, j)  
        
        # 计算各自商品的得分
        xui = i_bias + tf.reduce_sum(tf.multiply(u_vec, i_vec), 1, keep_dims=True)
        xuj = j_bias + tf.reduce_sum(tf.multiply(u_vec, j_vec), 1, keep_dims=True)
        xuij = xui-xuj
        #i+  j+ i>j
        auc = tf.reduce_mean(tf.to_float(xuij > 0))
        # norm
        l2_norm = tf.add_n([
              regulation_rate * tf.reduce_sum(tf.multiply(u_vec, u_vec)),
              regulation_rate * tf.reduce_sum(tf.multiply(i_vec, i_vec)),
              regulation_rate * tf.reduce_sum(tf.multiply(j_vec, j_vec)),
              bias_reg * tf.reduce_sum(tf.multiply(i_bias, i_bias)),
              bias_reg * tf.reduce_sum(tf.multiply(j_bias, j_bias)),
          ]) 
        #auc = tf.reduce_mean(tf.to_float(xuij > 0))
        # 目标函数 = tf.reduce_mean(tf.log(tf.sigmoid(xuij))) -  l2_norm   最大化
        # 那么求 l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(xuij))) 最小值即可
        # 将最大化问题转成最最小值问题
        bprloss =  l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(xuij))) 
        global_step = tf.Variable(0, trainable=False)
        # minimize 最大微分 求梯度
        train_op =  tf.train.AdamOptimizer().minimize(bprloss, global_step=global_step)  
    return u, i, j,auc, bprloss, train_op


4.batch训练评估效果

In [6]:
with tf.Graph().as_default(), tf.Session() as session:
    u, i, j, auc, bprloss, train_op = bpr_mf(user_count, item_count, 32)
    tf.global_variables_initializer().run()
    test_batch_data = generate_test_batch(test_data)
    _batch_bprloss = 0
    _batch_auc = 0
    for epoch in range(1, 2001):
        #
        batch_data = generate_train_batch(train_data)
        _auc,_bprloss, _train_op = session.run([auc, bprloss, train_op], 
                                feed_dict={u:batch_data[:,0], i:batch_data[:,1], j:batch_data[:,2]})
        _batch_bprloss += _bprloss
        _batch_auc += _auc
        
        if epoch%100 == 0:
            print ("epoch: ", epoch)
            print ("train_loss: ", _batch_bprloss / 100)
            print ("train_auc: ", _batch_auc / 100)
            _batch_bprloss = 0
            _batch_auc = 0
            #
            _auc, _bprloss = session.run([auc, bprloss],
                                    feed_dict={u:test_batch_data[:,0], i:test_batch_data[:,1], j:test_batch_data[:,2]}
                                )
            
            print("test_loss: ",_bprloss)
            print("test_auc: ",_auc)
        

epoch:  100
train_loss:  2.6148086261749266
train_auc:  0.504765625
test_loss:  34.37083
test_auc:  0.5084437
epoch:  200
train_loss:  1.8188515996932983
train_auc:  0.5390625
test_loss:  22.701008
test_auc:  0.5294702
epoch:  300
train_loss:  1.407367649078369
train_auc:  0.568671875
test_loss:  15.972724
test_auc:  0.56009936
epoch:  400
train_loss:  1.171027740240097
train_auc:  0.59921875
test_loss:  11.638198
test_auc:  0.5948675
epoch:  500
train_loss:  1.0230027079582213
train_auc:  0.628671875
test_loss:  8.8713255
test_auc:  0.61672187
epoch:  600
train_loss:  0.9290515553951263
train_auc:  0.6473046875
test_loss:  6.9396696
test_auc:  0.63195366
epoch:  700
train_loss:  0.8616610205173493
train_auc:  0.655078125
test_loss:  5.5858364
test_auc:  0.63327813
epoch:  800
train_loss:  0.8148200434446334
train_auc:  0.663203125
test_loss:  4.5690975
test_auc:  0.64139074
epoch:  900
train_loss:  0.7818043822050095
train_auc:  0.669921875
test_loss:  3.8697934
test_auc:  0.6428808
e